In [1]:
#!pip3 uninstall numpy
#!pip3 install numpy==1.19.2
#np.asarray(variable1, dtype="object")

In [2]:
!pip install keras

In [3]:
!pip install tensorflow

# Import Libraries

In [4]:
import warnings
warnings.filterwarnings('ignore')

import random
import json
import pickle
import numpy as np

import nltk
# English:
from nltk.stem import WordNetLemmatizer
# Arabic
from nltk.stem.isri import ISRIStemmer

from keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from keras.layers import Dense, Dropout
from keras.models import load_model

In [5]:
isstemmer = ISRIStemmer()

nltk.download('om-1.4')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Error loading om-1.4: Package 'om-1.4' not found in index
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Data Input

In [6]:
data_file = open("intents_arabic.json",encoding='utf-8', errors = 'ignore').read()
intents = json.loads(data_file)

In [7]:
intents

{'intents': [{'tag': 'greetings',
   'patterns': ['مرحبًا هناك',
    'مرحبًا',
    'السلام عليكم ',
    'ياو',
    'شو فيه',
    'كيف حالك؟',
    'مساء الخير',
    'هولا',
    'صباح الخير '],
   'responses': ['مرحبًا شكرًا للتحقق', 'مرحبًا، كيف يمكنني مساعدتك']},
  {'tag': 'goodbye',
   'patterns': ['وداعًا', 'باي', 'أراك لاحقًا'],
   'responses': ['أتمنى لك وقتًا سعيدًا، مرحبًا بك مرة أخرى', 'وداعًا']},
  {'tag': 'thanks',
   'patterns': ['شكرا',
    'حسنًا',
    'شكرًا لك',
    'شكرًا جزيلا',
    'ذلك مفيد',
    'رائع، شكرًا',
    'شكرًا على مساعدتي',
    'واو',
    'رائع'],
   'responses': ['سعيد بأن أساعد!',
    'في أي وقت!',
    'على الرحب والسعة',
    'بكل سرور']},
  {'tag': 'noanswer',
   'patterns': [''],
   'responses': ['عذرًا، لم أفهمك',
    'يرجى تقديم مزيد من المعلومات',
    'لست متأكدًا من فهمي لذلك']},
  {'tag': 'name1',
   'patterns': ['ما اسمك؟', 'من أنت؟'],
   'responses': ['أنا مجرد وكيل دردشة. أنا موجود فقط على الإنترنت',
    'أنا وكيل دردشة ']},
  {'tag': 'name',
 

# Data Preparation

In [8]:
words = []
classes = []
documents = []
ignore_words = ["?","!",".",","]

In [9]:
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        
        documents.append((w, intent["tag"]))
        
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

In [10]:
words

['مرحبًا',
 'هناك',
 'مرحبًا',
 'السلام',
 'عليكم',
 'ياو',
 'شو',
 'فيه',
 'كيف',
 'حالك؟',
 'مساء',
 'الخير',
 'هولا',
 'صباح',
 'الخير',
 'وداعًا',
 'باي',
 'أراك',
 'لاحقًا',
 'شكرا',
 'حسنًا',
 'شكرًا',
 'لك',
 'شكرًا',
 'جزيلا',
 'ذلك',
 'مفيد',
 'رائع،',
 'شكرًا',
 'شكرًا',
 'على',
 'مساعدتي',
 'واو',
 'رائع',
 'ما',
 'اسمك؟',
 'من',
 'أنت؟',
 'اسمي',
 'أنا',
 'أنا',
 'مين',
 'هل',
 'نتناول',
 'قهوة؟',
 'هل',
 'يمكنني',
 'أن',
 'أخرجك',
 'في',
 'موعد؟',
 'أحتاج',
 'مساعدة',
 'هل',
 'يمكنك',
 'مساعدتي',
 'أحتاجك',
 'كل',
 'ما',
 'أحتاجه',
 'هو',
 'أنت',
 'أريدك',
 'ما',
 'هو',
 'الذكاء',
 'الاصطناعي؟',
 'هل',
 'أنت',
 'ذكي؟',
 'هل',
 'لديك',
 'وعي؟',
 'هل',
 'أنت',
 'حكيم؟',
 'العنه',
 'بأي',
 'لغة',
 'تمت',
 'كتابتك؟',
 'تبدو',
 'مثل',
 'داتا',
 'أنت',
 'كيان',
 'لغوي',
 'اصطناعي',
 'أنت',
 'غير',
 'خالد',
 'هل',
 'أنت',
 'خالد؟',
 'أنت',
 'لا',
 'تفهم',
 'هل',
 'يمكن',
 'استنساخك',
 'لا',
 'يمكنك',
 'التحرك',
 'متى',
 'ستسير',
 'هل',
 'يمكنك',
 'المشي',
 'هل',
 'يمكنك',
 'التحر

In [11]:
classes

['greetings',
 'goodbye',
 'thanks',
 'noanswer',
 'name1',
 'name',
 'date',
 'fav',
 'need',
 'AI',
 'sentiment',
 'sapient',
 'abbr',
 'lang',
 'sound',
 'artificial',
 'imortal',
 'sense',
 'clone',
 'move',
 'bend',
 'laugh',
 'death',
 'robots',
 'stupid',
 'lie',
 'robotics',
 'machine',
 'fight',
 'chatbot',
 'chatterbox',
 'motormouth',
 'ratchet',
 'body',
 'business',
 'programming',
 'hobby',
 'idea',
 'shoe',
 'robotss',
 'computer',
 'os',
 'comp',
 'hardware',
 'cramped',
 'program',
 'breathe',
 'control',
 'malfunction',
 'usage',
 'who',
 'bot1',
 'events',
 'do',
 'wt']

In [12]:
documents

[(['مرحبًا', 'هناك'], 'greetings'),
 (['مرحبًا'], 'greetings'),
 (['السلام', 'عليكم'], 'greetings'),
 (['ياو'], 'greetings'),
 (['شو', 'فيه'], 'greetings'),
 (['كيف', 'حالك؟'], 'greetings'),
 (['مساء', 'الخير'], 'greetings'),
 (['هولا'], 'greetings'),
 (['صباح', 'الخير'], 'greetings'),
 (['وداعًا'], 'goodbye'),
 (['باي'], 'goodbye'),
 (['أراك', 'لاحقًا'], 'goodbye'),
 (['شكرا'], 'thanks'),
 (['حسنًا'], 'thanks'),
 (['شكرًا', 'لك'], 'thanks'),
 (['شكرًا', 'جزيلا'], 'thanks'),
 (['ذلك', 'مفيد'], 'thanks'),
 (['رائع،', 'شكرًا'], 'thanks'),
 (['شكرًا', 'على', 'مساعدتي'], 'thanks'),
 (['واو'], 'thanks'),
 (['رائع'], 'thanks'),
 ([], 'noanswer'),
 (['ما', 'اسمك؟'], 'name1'),
 (['من', 'أنت؟'], 'name1'),
 (['اسمي'], 'name'),
 (['أنا'], 'name'),
 (['أنا', 'مين'], 'name'),
 (['هل', 'نتناول', 'قهوة؟'], 'date'),
 (['هل', 'يمكنني', 'أن', 'أخرجك', 'في', 'موعد؟'], 'date'),
 (['أحتاج', 'مساعدة'], 'fav'),
 (['هل', 'يمكنك', 'مساعدتي'], 'fav'),
 (['أحتاجك'], 'need'),
 (['كل', 'ما', 'أحتاجه', 'هو', 'أنت']

In [13]:
processed_words = []
for w in words:
    stemmed_word = isstemmer.stem(w)
    
    if w not in ignore_words:
        processed_words.append(stemmed_word)
        
words = sorted(list(set(processed_words)))
classes = sorted(list(classes))

In [14]:
words

['اذا',
 'ارد',
 'ارك',
 'استنساخك',
 'اسم',
 'اصطناعي؟',
 'امت',
 'امل',
 'ان',
 'انا',
 'انت',
 'انك',
 'بأن',
 'بأي',
 'باي',
 'بدو',
 'بضح',
 'بك',
 'بوت',
 'تحب',
 'تحر',
 'تسر',
 'تكون',
 'تمت',
 'ثرثار',
 'ثرثر',
 'جاب',
 'جدد',
 'جزل',
 'جسد',
 'جهز',
 'حاج',
 'حالك؟',
 'حجم',
 'حدث',
 'حذك',
 'حسن',
 'حكم',
 'حكيم؟',
 'خاص',
 'خالد؟',
 'خبر',
 'خدم',
 'خرج',
 'خصة',
 'خلد',
 'خلف',
 'خير',
 'دتا',
 'ذك؟',
 'ذكء',
 'ذلك',
 'رئع',
 'رائع،',
 'رحب',
 'رمج',
 'رنمج',
 'روبو',
 'سعد',
 'سلم',
 'سمح',
 'شعر',
 'شغل',
 'شكر',
 'شو',
 'صبح',
 'صحح',
 'صطناع',
 'صندوق',
 'ضيق',
 'عطل',
 'عل؟',
 'على',
 'علي',
 'عمل',
 'عنه',
 'غبة',
 'غبي',
 'غير',
 'فضل',
 'فعل',
 'فكر',
 'فهم',
 'في',
 'فيد',
 'فيه',
 'قتل',
 'قدم',
 'قهوة؟',
 'كتابتك؟',
 'كذب',
 'كل',
 'كمبيوتر',
 'كيف',
 'كين',
 'لا',
 'لحق',
 'لدك',
 'لغة',
 'لغي',
 'لك',
 'لها',
 'لي',
 'ما',
 'متى',
 'مثل',
 'محادثة؟',
 'مساء',
 'مشي',
 'مك؟',
 'من',
 'منتجك؟',
 'موت',
 'مين',
 'نت؟',
 'نظم',
 'نفس',
 'نوع',
 'نول',
 'هدف',
 'هذ

In [15]:
classes

['AI',
 'abbr',
 'artificial',
 'bend',
 'body',
 'bot1',
 'breathe',
 'business',
 'chatbot',
 'chatterbox',
 'clone',
 'comp',
 'computer',
 'control',
 'cramped',
 'date',
 'death',
 'do',
 'events',
 'fav',
 'fight',
 'goodbye',
 'greetings',
 'hardware',
 'hobby',
 'idea',
 'imortal',
 'lang',
 'laugh',
 'lie',
 'machine',
 'malfunction',
 'motormouth',
 'move',
 'name',
 'name1',
 'need',
 'noanswer',
 'os',
 'program',
 'programming',
 'ratchet',
 'robotics',
 'robots',
 'robotss',
 'sapient',
 'sense',
 'sentiment',
 'shoe',
 'sound',
 'stupid',
 'thanks',
 'usage',
 'who',
 'wt']

In [16]:
len(documents)

101

In [17]:
len(words)

137

# Data Creation

In [18]:
training = []
output_empty = [0] * len(classes) 

In [19]:
training = []
output_empty = [0] * len(classes)

for doc in documents:
    #print("doc",doc)
    bag = []
    pattern_words = doc[0]
    #print("pattern_words_1", pattern_words)
    
    pattern_words = [isstemmer.stem(word) for word in pattern_words]
    #print("pattern_words_2", pattern_words)
    '''
    for w in words:
        if w in pattern_words:
            bag.append(1)
        else:
            bag.append(0)'''
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    #print("bag_1",bag)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])]=1
    #print("output_row",output_row)
    
    training.append([bag, output_row])

In [20]:
print(training)

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0,

In [21]:
random.shuffle(training)
training = np.array(training,dtype=object)
#training = np.array(training,dtype=object)

In [22]:
training

array([[list([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]),
        list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
       [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [25]:
train_x = list(training[:,0])
train_y = list(training[:,1])

In [27]:
print(train_x)

[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [28]:
print(train_y)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0